In [17]:
from src import environments
import nest_asyncio
nest_asyncio.apply()

from gym_derk.envs import DerkEnv
import gym_derk
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
import pygad
from src.environments import EnvPerso

In [2]:
%load_ext autoreload
%autoreload 2

In [18]:
pygad.__version__

'3.1.0'

In [22]:
from pygad import kerasga

AttributeError: module 'pygad' has no attribute 'kerasga'

In [24]:
model = keras.Sequential()
model.add(layers.Dense(60, activation="relu", input_shape=(64,)))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(15, activation='tanh'))

In [27]:
ga = kerasga.KerasGA(model=model,
                                 num_solutions=10)

In [28]:
def fitness_func(ga_instance, solution, solution_idx):
    global model, env
    reward= {
    "damageEnemyStatue": 4,
    "damageEnemyUnit": 2,
    "killEnemyStatue": 4,
    "killEnemyUnit": 2,
    "healFriendlyStatue": 1,
    "healTeammatel": 2,
    "healTeammate2": 2,
    "timeSpentHomeBase": 0,
    "timeSpentHomeTerritory": 0,
    "timeSpentAwayTerritory": 0,
    "timeSpentAwayBase": 0,
    "damageTaken": -1,
    "friendlyFire": -1,
    "healEnemy": -1,
    "fallDamageTaken": -10,
    "statueDamageTaken": 0,
    "manualBonus": 0,
    "victory": 100,
    "loss": -100,
    "tie": 0,
    "teamSpirit": 0.5,
    "timeScaling": 1,
}
    
    env = EnvPerso(reward_function = reward)
    obs = env.reset()

    fitness = 0
    done = False
    while not done:
        action = model.predict(obs)
        obs, reward, done, info = env.step(action)
        fitness += reward
    env.close()
    return fitness

In [29]:
num_generations = 100
num_parents_mating  = 2
initial_population = ga.population_weights

In [30]:
def on_generation(ga_instance):
    print("===========================================")

In [31]:
instance = pygad.GA(num_generations=num_generations,
                    num_parents_mating=num_parents_mating,
                    initial_population=initial_population,
                    fitness_func=fitness_func,
                    on_generation=on_generation)
                    

In [32]:
instance.run()

in user code:

    File "/home/trist/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/home/trist/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/trist/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/home/trist/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/home/trist/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "

AssertionError: 

In [33]:
env.close()

NameError: name 'env' is not defined

In [ ]:
# class Agent:
#     def __init__(self, shape_of_model: tuple):
#         nb_outputs = 15
#         self.model = keras.models.Sequential()
#         self.model.add(layers.Input(shape=(64,)))
#         for n in shape_of_model:
#             self.model.add(layers.Dense(units=n, activation='relu'))
#         self.model.add(layers.Dense(units=nb_outputs, activation='linear'))
    
#     def action(self, state: np.ndarray):
#         prediction = self.model.predict(state)
#         # min max scaling
        
#         return self.model.predict(state)
    


In [ ]:
# class AgentManager:
#     def __init__(self, env, number_of_agents):
#         self.env = env
#         self.agents = [Agent(shape_of_model=(15,15)) for i in range(number_of_agents)]
            
#     def take_action(self, observation):
#         return [agent.action(observation) for agent in self.agents]   
        
        